In [1]:
import pandas as pd
from datasets import Dataset
import numpy as np
import torch
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from scipy.spatial.distance import cosine
from sentence_transformers import SentenceTransformer
import evaluate

c:\Anaconda\envs\Cuda\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Anaconda\envs\Cuda\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Anaconda\envs\Cuda\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
c:\Anaconda\envs\Cuda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
Train_Data_Frame = pd.read_csv('../../../Datasets/Validation Merged/Dataset.csv')
Train_Data_Frame

,paragraph,summary
0,إيمان عريقات بي بي سي - بيت لحم الرياضة غيرت ح...,بدأت قصة الفلسطيني نور إسماعيل ذو الإعاقة في ع...
1,قوات الحشد الشعبي في المقدادية وبين المصدر أن ...,أفاد مصدر أمني في محافظة ديالى شمال شرقي بغداد...
2,"المجلد الأول من مذكرات باراك أوباما ""أرض الميع...",أعرب الرئيس الأمريكي السابق ، باراك أوباما ، ف...
3,تستمر عمليات إنقاذ الحيتان على شواطيء الخليج ا...,لسوء الحظ ، ساء الوضع على شواطئ نيوزيلندا النا...
4,الجدل حول موازنة الاتحاد الأوروبي للأعوام السب...,يبدأ قادة الاتحاد الأوروبي مفاوضات شاملة في بر...
...,...,...
2061,أرامكو عامل في شركة أرامكو يرتدي ما يشبه جهاز ...,كانت هناك موجة من القلق وخيبة الأمل التي تم ال...
2062,يتهم كثيرون مودي بالضلوع في اعمال الشغب التي ا...,وصل رئيس الوزراء الهندي ناريندرا مودي إلى الول...
2063,يسعى الرئيس الروسي فلاديمير بوتين إلى تغيير ال...,صرح الرئيس الروسي فلاديمير بوتين في وقت سابق م...
2064,مئات المتظاهرين تجمهروا خارج مبنى نقابة الصحاف...,ووقعت اشتباكات بين قوات الأمن المصرية ومتظاهري...


In [3]:
Train_Data_Frame = Dataset.from_pandas(Train_Data_Frame)

In [4]:
Train_Dataset_Paragraphs_Column_Name = 'paragraph'
Train_Dataset_Summaries_Column_Name = 'summary'

In [5]:
Train_Data_Frame = Train_Data_Frame.train_test_split(test_size=0.2)

In [6]:
Train_Data_Frame["train"][0]

{'paragraph': 'بيع العقار الذي يضم منزل غليمبتون والذي يعود إلى القرن الثامن عشر تم في فبراير/ شباط الماضي. نبدأ بصحيفة الغارديان التي خصصت تقريرا لقيام الأمير السعودي بندر بن سلطان ببيع عقار ريفي كبير في منطقة كوتسوولد في المملكة المتحدة لملك البحرين مقابل أكثر من 120 مليون جنيه استرليني. وباع الأمير بندر بن سلطان وهو أحد كبار الشخصيات الملكية السعودية والسفير السابق في الولايات المتحدة والمدير العام السابق لوكالة المخابرات السعودية، عقارات غليمبتون بارك لعائلة ملك البحرين حمد بن عيسى آل خليفة ونجله ولي العهد، سلمان بن حمد بن عيسى آل خليفة. وأفاد المقال بأن بيع العقار الذي يضم منزل غليمبتون الذي يعود إلى القرن الثامن عشر، بالإضافة إلى 39 كوخا وكنيسة أبرشية نورماندية و167 فدانا من الحدائق، قد تم في فبراير/ شباط الماضي. ولفت إلى أن "الأمير بندر (72 عاما) حقق أرباحا ضخمة من عملية البيع، حيث اشترى العقار في التسعينيات مقابل حوالي 8 ملايين جنيه إسترليني من قطب العقارات الأسترالي آلان بوند الذي سُجن لاحقا بسبب أكبر عملية احتيال في أستراليا". مواضيع قد تهمك نهاية وأشار المقال إلى أن "فاحشي ا

In [7]:
Model_Name = "facebook/mbart-large-50"
Tokenizer = AutoTokenizer.from_pretrained(Model_Name)
Model = AutoModelForSeq2SeqLM.from_pretrained(Model_Name)
Semantic_Model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [8]:
Prefix = "summarize: "
Paragraph_Max_Length = 1000
Summary_Max_Length = 300

In [9]:
def Tokenization(Records):
    Paragraphs = [Prefix + Paragraph for Paragraph in Records[Train_Dataset_Paragraphs_Column_Name]]
    Final_Tokenized_Inputs = Tokenizer(Paragraphs, max_length=Paragraph_Max_Length, truncation=True, padding="longest", return_tensors="pt")

    Labels = Tokenizer(Records[Train_Dataset_Summaries_Column_Name], max_length=Summary_Max_Length, truncation=True, padding="longest", return_tensors="pt")

    Final_Tokenized_Inputs["labels"] = Labels["input_ids"]
    return Final_Tokenized_Inputs

In [10]:
Tokenized_Dataset = Train_Data_Frame.map(Tokenization, batched=True)

Map:   0%|          | 0/1652 [00:00<?, ? examples/s]

In [11]:
Data_Collator = DataCollatorForSeq2Seq(tokenizer=Tokenizer, model=Model_Name)

In [12]:
Rouge = evaluate.load("rouge")

In [13]:
def Compute_Metrics(Eval_Pred):
    Predictions, Labels = Eval_Pred
    Decoded_Preds = Tokenizer.batch_decode(Predictions, skip_special_tokens=True)
    Labels = np.where(Labels != -100, Labels, Tokenizer.pad_token_id)
    Decoded_Labels = Tokenizer.batch_decode(Labels, skip_special_tokens=True)

    similarity_scores = []
    for pred, label in zip(Decoded_Preds, Decoded_Labels):
        pred = pred[len(Prefix):]
        embeddings = Semantic_Model.encode([pred, label], convert_to_tensor=True)
        similarity_score = 1 - cosine(embeddings[0].cpu(), embeddings[1].cpu())
        similarity_scores.append(similarity_score)

    Result = Rouge.compute(predictions=Decoded_Preds, references=Decoded_Labels, use_stemmer=True)

    Prediction_Lens = [np.count_nonzero(Pred != Tokenizer.pad_token_id) for Pred in Predictions]
    Result["gen_len"] = np.mean(Prediction_Lens)
    Result["semantic_similarity"] = np.mean(similarity_scores)

    print("Decoded Predictions:", Decoded_Preds)
    print("Decoded Labels:", Decoded_Labels)

    return {k: round(v, 4) for k, v in Result.items()}

In [14]:
training_args = Seq2SeqTrainingArguments(
    output_dir="Check_Points_Output",
    evaluation_strategy="epoch",
    per_device_train_batch_size=3,
    per_device_eval_batch_size=3,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
)

trainer = Seq2SeqTrainer(
    model=Model,
    args=training_args,
    train_dataset=Tokenized_Dataset["train"],
    eval_dataset=Tokenized_Dataset["test"],
    tokenizer=Tokenizer,
    data_collator=Data_Collator,
    compute_metrics=Compute_Metrics,
)

In [15]:
trainer.train()

c:\Anaconda\envs\Cuda\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
 23%|██▎       | 500/2204 [3:37:11<12:19:07, 26.03s/it]

{'loss': 1.588, 'learning_rate': 3.865698729582577e-05, 'epoch': 0.91}


                                                       
 25%|██▌       | 551/2204 [4:40:25<10:35:51, 23.08s/it]

Decoded Predictions: ['في المملكة المتحدة ، هناك العديد من الاستفسارات حول قلة النوم ، والتي يمكن أن يؤدي إلى زيادة الوزن. بالإضافة إلى ذلك ، هناك مخاوف بشأن زيادة الوزن. ومع ذلك ، من المهم ملاحظة أن قلة النوم يمكن أن يؤدي إلى زيادة الوزن.', 'نشرت صحيفة التايمز مقالاً بعنوان "الجهاديون تحضروا لخسائرهم في العراق وسوريا بتنفيذ هجمات في الغرب". جاء هذا المقال للتعبير عن قلقها العميق لتنظيم الدولة الإسلامية في العراق وسوريا.', 'على منصات التواصل الاجتماعي ، مثل Facebook و Twitter ، هناك مخاوف بشأن تفشي فيروس كورونا. بالإضافة إلى ذلك ، هناك مخاوف بشأن التأثير المحتمل للفيروس على الصحة العامة.', 'أبلغت مايا ، البالغة من العمر 28 عامًا ، عن اعتقالها في طوكيو ، بتهمة التهرب الضريبي من قبل أفراد عائلتها. بالإضافة إلى ذلك ، أبلغت مايا ، البالغة من العمر 28 عامًا ، عن اعتقالها بتهم التهرب الضريبي.', 'لا يزال مخيم الزعتري ، المعروف أيضًا باسم "مخيم الخمس نجوم" ، محظوظًا للاجئين السوريين من مختلف مناطق الأردن. ومع ذلك ، فقد حظي مخيم الزعتري باهتمام كبير من قبل العديد من اللاجئين السوريين ، الذين يق

 45%|████▌     | 1000/2204 [7:55:28<8:32:48, 25.56s/it]  

{'loss': 0.715, 'learning_rate': 2.7313974591651543e-05, 'epoch': 1.81}


                                                       
 50%|█████     | 1102/2204 [9:22:19<7:07:12, 23.26s/it]

Decoded Predictions: ['هل تعتقد الحكومة البريطانية أن من أجل صحة أفضل في المملكة المتحدة ، هناك حاجة إلى تخفيض مستويات البدانة؟ بالإضافة إلى ذلك ، هناك حملة تهدف إلى تشجيع الأفراد على ممارسة الرياضة ، مثل حملة "خمس حصص في اليوم" من الخضروات والفاكهة ، وحملة "عشرة آلاف خطوة في اليوم" للتشجيع على التريض.', 'غطت الصحف البريطانية ، صباح اليوم الاثنين ، مواضيع مختلفة قد تهم القارئ العربي. وتشمل هذه الخسارة المأساوية لتنظيم الدولة الإسلامية في العراق وسوريا ، فضلاً عن تبرعات مالية لزوجة سابقة لجورج غالاوي لدعم طفلة عراقية مريضة بالسرطان. بالإضافة إلى ذلك ، كانت هناك تقارير عن تبرعات مالية جمعتها امرأة عراقية لجورج غالاوي لدعم طفلة عراقية.', 'على منصات التواصل الاجتماعي المختلفة ، بما في ذلك منصات الجيل الخامس من الهواتف المحمولة (الجيل الخامس) وشبكات الجيل الرابع (الجيل الخامس) ، هناك العديد من الاستفسارات حول العوامل الكامنة وراء تفشي فيروس كورونا. بالإضافة إلى ذلك ، هناك العديد من الفيديوهات التي تشير إلى أن الجيل الخامس من الهواتف المحمولة (الجيل الرابع) قد يكون مرتبطًا بشبكات 5G.', 'أبلغ

 65%|██████▌   | 1440/2204 [11:48:39<5:32:51, 26.14s/it]  

In [ ]:
Model_Save_Path = 'mBart.pt'
torch.save(Model.state_dict(), Model_Save_Path)